In [ ]:
import pandas as pd
import transformers
import torch
import random
import pandas as pd
import random
import string
import numpy as np
import unidecode
from unidecode import unidecode
from wordcloud import STOPWORDS
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer, BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, PreTrainedTokenizerFast
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
import torch
transformers.logging.set_verbosity_debug()
from loguru import logger
logger.add("logging/finnlp_bert_large_experiment_1.log", rotation="500 MB")

In [3]:
BATCH_SIZE = 15

In [4]:
logger.debug("<<<<<<<<<< INICIOU >>>>>>>>>>>")
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")

2024-03-01 19:42:50.913 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>


In [5]:
brc.head(2)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
0,1,DPT_25,2020-12-23,2128,36560,2020-12-23,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,656,67.0,127.0,COMUNICADO,771984,https://www.bcb.gov.br/estabilidadefinanceira/...
1,1,DPT_25,2020-12-28,2129,36569,2020-12-28,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,2110,156.0,322.0,COMUNICADO,772973,https://www.bcb.gov.br/estabilidadefinanceira/...


In [6]:
brc.keys()

Index(['class', 'department', 'entry_date', 'general_id',
       'normative_identifier', 'publication_date', 'regulatory_authority',
       'subject', 'subject_length', 'subject_unique_words', 'subject_words',
       'text', 'text_length', 'text_unique_words', 'text_words', 'type',
       'unique_document_id', 'url'],
      dtype='object')

In [7]:
brc.count()

class                   61864
department              61864
entry_date              61864
general_id              61864
normative_identifier    61864
publication_date        61864
regulatory_authority    61864
subject                 61864
subject_length          61864
subject_unique_words    61864
subject_words           61864
text                    61864
text_length             61864
text_unique_words       61864
text_words              61864
type                    61864
unique_document_id      61864
url                     61864
dtype: int64

In [8]:
brc = brc[brc.text_words >= 50]
brc.count()

class                   60836
department              60836
entry_date              60836
general_id              60836
normative_identifier    60836
publication_date        60836
regulatory_authority    60836
subject                 60836
subject_length          60836
subject_unique_words    60836
subject_words           60836
text                    60836
text_length             60836
text_unique_words       60836
text_words              60836
type                    60836
unique_document_id      60836
url                     60836
dtype: int64

In [9]:
sel_cols = [column for column, is_type in (brc.dtypes=="object").items() if is_type]
sel_cols

['department',
 'entry_date',
 'normative_identifier',
 'publication_date',
 'regulatory_authority',
 'subject',
 'text',
 'type',
 'url']

In [10]:
brc['class'] = brc['class'].astype(str)

In [11]:
brc = brc.rename(columns={"class":"label"})

In [12]:
brc.count()

label                   60836
department              60836
entry_date              60836
general_id              60836
normative_identifier    60836
publication_date        60836
regulatory_authority    60836
subject                 60836
subject_length          60836
subject_unique_words    60836
subject_words           60836
text                    60836
text_length             60836
text_unique_words       60836
text_words              60836
type                    60836
unique_document_id      60836
url                     60836
dtype: int64

In [13]:
class Model_BERT():
    
    def model_path_def(self, name_model):
        if name_model == "BERTIMBAU_LARGE":
            model_path = MODEL_PATH
        elif name_model == "XXX":
            model_path = MODEL_PATH_NEW
        else:
            print("Nome de modelo inválido")
            model_path = "None"
            
        return model_path

    def tokenizer_model(self, model_path, df, token):
        tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

        # Tokenizing the training data
        encoded_data_train = tokenizer.batch_encode_plus(
            df[df.data_type=='train'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True, 
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )

        # Tokenizing test data
        encoded_data_val = tokenizer.batch_encode_plus(
            df[df.data_type=='val'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True,
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )


        input_ids_train = encoded_data_train['input_ids']
        attention_masks_train = encoded_data_train['attention_mask']
        labels_train = torch.tensor(df[df.data_type=='train'].label_num.values)

        input_ids_val = encoded_data_val['input_ids']
        attention_masks_val = encoded_data_val['attention_mask']
        labels_val = torch.tensor(df[df.data_type=='val'].label_num.values)

        dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
        dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
        
        return(dataset_train, dataset_val)

    def setup_model(self, model_path, dataset_train, dataset_val, epochs):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        batch_size = BATCH_SIZE
        dataloader_train = DataLoader(dataset_train, 
                                      sampler=RandomSampler(dataset_train), 
                                      batch_size=batch_size)
        dataloader_validation = DataLoader(dataset_val, 
                                           sampler=SequentialSampler(dataset_val), 
                                           batch_size=batch_size)
        optimizer = AdamW(model.parameters(),
                          lr=1e-5, 
                          eps=1e-8)

        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=100,
                                                    num_training_steps=len(dataloader_train)*epochs)
        return(model, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device)


    def f1_score_func(self, preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        scr = f1_score(labels_flat, preds_flat, average='binary')
        logger.info(f"f1_score_binary: {scr}")
        return scr

    def accuracy_per_class(self, preds, labels):
        label_dict_inverse = {v: k for k, v in LABEL_DICT.items()}

        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            print(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
            logger.info(f'Class: {label_dict_inverse[label]}')
            logger.info(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            logger.info(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
        
    def evaluate(self, model, dataloader_val, device):

        model.eval()

        loss_val_total = 0
        predictions, true_vals = [], []

        for batch in dataloader_val:

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            with torch.no_grad():        
                outputs = model(**inputs)

            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        loss_val_avg = loss_val_total/len(dataloader_val) 

        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)

        logger.info(f"loss_val_avg, predictions, true_vals: {loss_val_avg}, {predictions}, {true_vals}")

        return loss_val_avg, predictions, true_vals

    def results(self, model_path, model_name, epochs, dataloader_validation, device, token):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        model.to(device)

        model.load_state_dict(torch.load(f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epochs}.model', map_location=torch.device('cpu')))

        _, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
        Model_BERT().accuracy_per_class(predictions, true_vals)
        from sklearn import metrics
        preds_flat = np.argmax(predictions, axis=1).flatten()
        metrics_report = metrics.classification_report(true_vals,preds_flat, output_dict=True)
        logger.info(metrics_report)
        
        return metrics_report
    

    def training_model(self, model, model_name, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device, token):
        seed_val = 1
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        epoch_array = []
        f1_array = []
        train_loss_array = []
        val_loss_array = []

        for epoch in tqdm(range(1, epochs+1)):

            model.train()

            loss_train_total = 0

            progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
            for batch in progress_bar:

                model.zero_grad()

                batch = tuple(b.to(device) for b in batch)

                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[2],
                         }       

                outputs = model(**inputs)

                loss = outputs[0]
                loss_train_total += loss.item()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()
                scheduler.step()

                progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


            torch.save(model.state_dict(),f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}.model')
            model.save_pretrained(f"{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}")
            tqdm.write(f'\nEpoch {epoch}')
            print(f'\nEpoch {epoch}')
            logger.info(f'\nEpoch {epoch}')

            loss_train_avg = loss_train_total/len(dataloader_train)            
            tqdm.write(f'Training loss: {loss_train_avg}')
            logger.info(f'Training loss: {loss_train_avg}')

            val_loss, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
            val_f1 = Model_BERT().f1_score_func(predictions, true_vals)
            tqdm.write(f'Validation loss: {val_loss}')
            print(f'Validation loss: {val_loss}')
            logger.info(f'Validation loss: {val_loss}')
            
            tqdm.write(f'F1 Score (Weighted/Binary): {val_f1}')
            print(f'F1 Score (Weighted/Binary): {val_f1}')
            logger.info(f'F1 Score (Weighted/Binary): {val_f1}')
            
            epoch_array.append(epoch)
            f1_array.append(val_f1)
            val_loss_array.append(val_loss)
            train_loss_array.append(loss_train_avg)

        return(epoch_array, f1_array, val_loss_array, train_loss_array)
    
    def read_df(self, df):
        df['label_num'] = df.label.replace(LABEL_DICT)
        X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label_num.values, 
                                                  test_size=0.4, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
        df['data_type'] = ['not_set']*df.shape[0]
        df.loc[X_train, 'data_type'] = 'train'
        df.loc[X_val, 'data_type'] = 'val'
        
        return(df)

    def call_setup(self, model_name, df, token, epochs):
        df = Model_BERT().read_df(df)
        model_path = Model_BERT().model_path_def(model_name)
        print(model_path)
        dataset_train, dataset_val = Model_BERT().tokenizer_model(model_path, df, token)
        model, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device = Model_BERT().setup_model(model_path, dataset_train, dataset_val, epochs)
        epoch_array, f1_array, val_loss_array, train_loss_array = Model_BERT().training_model(model, model_name, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device, token)
        df_performa = pd.DataFrame()
        df_performa["epoch"] = epoch_array
        df_performa["f1"] = f1_array
        df_performa["val_loss"] = val_loss_array
        df_performa["train_loss"] = train_loss_array
        metrics_report = Model_BERT().results(model_path,model_name,epochs,dataloader_val, device, token)
        logger.info(f'Metric Report Call Setup: {metrics_report}')
        
        return metrics_report, df_performa

In [15]:
departments = ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']

In [16]:
brc.keys()

Index(['label', 'department', 'entry_date', 'general_id',
       'normative_identifier', 'publication_date', 'regulatory_authority',
       'subject', 'subject_length', 'subject_unique_words', 'subject_words',
       'text', 'text_length', 'text_unique_words', 'text_words', 'type',
       'unique_document_id', 'url'],
      dtype='object')

In [18]:
def cleaning(df,column):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    return df

In [ ]:
for department in departments:
    logger.debug(f"Department: {department}")
    df_modelagem = brc[brc.department == department]

    df_modelagem = df_modelagem.dropna()
    df_modelagem = df_modelagem.reset_index(drop=True)
    df_modelagem = df_modelagem.rename(columns={"text":"content"})

    df_modelagem = cleaning(df_modelagem,"content")

    new_content = []
    for texto in df_modelagem.content:
        new_content.append(texto.replace("[BUSINESS NEED]", ""))
    df_modelagem["new_content"] = new_content
       
    LABEL_DICT = {'0': 0, '1': 1}

    MODEL_SAVE_PATH = "notebooks/testes/scoring"
    BERTIMBAU_LARGE = "dataset/bert-large-portuguese-cased"
    BERTIMBAU_TOKENIZER = "tokenizer/BERTimbau_Tokenizer"

    MODEL_PATH = BERTIMBAU_LARGE
    MODEL_PATH_NEW = BERTIMBAU_LARGE

    tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

    logger.debug(f"Department count samples: {df_modelagem.count()}")
    logger.debug(f"Department count samples class 1: {df_modelagem[df_modelagem.label == '1'].count()}")
    logger.debug(f"Department count samples class 0: {df_modelagem[df_modelagem.label == '0'].count()}")
    

    Model_BERT().call_setup("BERTIMBAU_LARGE", df_modelagem, 512, 5)

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2023-10-13 11:51:13.824 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>
# 2023-10-13 11:51:23.874 | DEBUG    | __main__:<module>:2 - Department: DPT_25
# 2023-10-13 11:51:38.316 | DEBUG    | __main__:<module>:37 - Department count samples: label                   4870
# department              4870
# entry_date              4870
# general_id              4870
# normative_identifier    4870
# publication_date        4870
# regulatory_authority    4870
# subject                 4870
# subject_length          4870
# subject_unique_words    4870
# subject_words           4870
# content                 4870
# text_length             4870
# text_unique_words       4870
# text_words              4870
# type                    4870
# unique_document_id      4870
# url                     4870
# new_content             4870
# dtype: int64
# 2023-10-13 11:51:38.321 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   819
# department              819
# entry_date              819
# general_id              819
# normative_identifier    819
# publication_date        819
# regulatory_authority    819
# subject                 819
# subject_length          819
# subject_unique_words    819
# subject_words           819
# content                 819
# text_length             819
# text_unique_words       819
# text_words              819
# type                    819
# unique_document_id      819
# url                     819
# new_content             819
# dtype: int64
# 2023-10-13 11:51:38.328 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   4051
# department              4051
# entry_date              4051
# general_id              4051
# normative_identifier    4051
# publication_date        4051
# regulatory_authority    4051
# subject                 4051
# subject_length          4051
# subject_unique_words    4051
# subject_words           4051
# content                 4051
# text_length             4051
# text_unique_words       4051
# text_words              4051
# type                    4051
# unique_document_id      4051
# url                     4051
# new_content             4051
# dtype: int64
# 2023-10-13 12:00:40.936 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-13 12:00:40.937 | INFO     | __main__:training_model:214 - Training loss: 0.2935211061905974
# 2023-10-13 12:01:38.791 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.23021956456538575, [[-1.7027459  2.254149 ]
 
# 2023-10-13 12:01:38.795 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7907801418439716
# 2023-10-13 12:01:38.796 | INFO     | __main__:training_model:220 - Validation loss: 0.23021956456538575
# 2023-10-13 12:01:38.797 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7907801418439716
# 2023-10-13 12:06:11.525 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-13 12:06:11.526 | INFO     | __main__:training_model:214 - Training loss: 0.17974417964712933
# 2023-10-13 12:07:09.391 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.1924191405244458, [[-2.1077762  2.4839356]
 
# 2023-10-13 12:07:09.395 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8166666666666668
# 2023-10-13 12:07:09.397 | INFO     | __main__:training_model:220 - Validation loss: 0.1924191405244458
# 2023-10-13 12:07:09.398 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8166666666666668
# 2023-10-13 12:11:42.424 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-13 12:11:42.425 | INFO     | __main__:training_model:214 - Training loss: 0.13468760723510806
# 2023-10-13 12:12:40.250 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.1734121194557072, [[-2.4308465  2.8212116]
 
# 2023-10-13 12:12:40.254 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8206686930091185
# 2023-10-13 12:12:40.256 | INFO     | __main__:training_model:220 - Validation loss: 0.1734121194557072
# 2023-10-13 12:12:40.257 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8206686930091185
# 2023-10-13 12:17:13.154 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-13 12:17:13.155 | INFO     | __main__:training_model:214 - Training loss: 0.09639057335563195
# 2023-10-13 12:18:11.041 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.18175953362325134, [[-2.7674682  3.142334 ]
 
# 2023-10-13 12:18:11.045 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8356807511737088
# 2023-10-13 12:18:11.046 | INFO     | __main__:training_model:220 - Validation loss: 0.18175953362325134
# 2023-10-13 12:18:11.047 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8356807511737088
# 2023-10-13 12:22:43.591 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-13 12:22:43.592 | INFO     | __main__:training_model:214 - Training loss: 0.07042238765652889
# 2023-10-13 12:23:41.473 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.19972737598828433, [[-2.8547685  3.2312648]
 
# 2023-10-13 12:23:41.476 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8274760383386581
# 2023-10-13 12:23:41.477 | INFO     | __main__:training_model:220 - Validation loss: 0.19972737598828433
# 2023-10-13 12:23:41.478 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8274760383386581
# 2023-10-13 12:24:43.907 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.19972737598828433, [[-2.8547685  3.2312648]
 
# 2023-10-13 12:24:43.908 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-13 12:24:43.909 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 1581/1620
# 2023-10-13 12:24:43.910 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.975925925925926

# 2023-10-13 12:24:43.911 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-13 12:24:43.911 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 259/328
# 2023-10-13 12:24:43.912 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.7896341463414634

# 2023-10-13 12:24:43.921 | INFO     | __main__:results:157 - {'0': {'precision': 0.9581818181818181, 'recall': 0.975925925925926, 'f1-score': 0.9669724770642202, 'support': 1620}, '1': {'precision': 0.8691275167785235, 'recall': 0.7896341463414634, 'f1-score': 0.8274760383386581, 'support': 328}, 'accuracy': 0.944558521560575, 'macro avg': {'precision': 0.9136546674801709, 'recall': 0.8827800361336947, 'f1-score': 0.8972242577014391, 'support': 1948}, 'weighted avg': {'precision': 0.9431870487463558, 'recall': 0.944558521560575, 'f1-score': 0.9434843703383555, 'support': 1948}}
# 2023-10-13 12:24:43.924 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9581818181818181, 'recall': 0.975925925925926, 'f1-score': 0.9669724770642202, 'support': 1620}, '1': {'precision': 0.8691275167785235, 'recall': 0.7896341463414634, 'f1-score': 0.8274760383386581, 'support': 328}, 'accuracy': 0.944558521560575, 'macro avg': {'precision': 0.9136546674801709, 'recall': 0.8827800361336947, 'f1-score': 0.8972242577014391, 'support': 1948}, 'weighted avg': {'precision': 0.9431870487463558, 'recall': 0.944558521560575, 'f1-score': 0.9434843703383555, 'support': 1948}}
# 2023-10-13 12:24:43.928 | DEBUG    | __main__:<module>:2 - Department: DPT_3
# 2023-10-13 12:25:04.366 | DEBUG    | __main__:<module>:37 - Department count samples: label                   2478
# department              2478
# entry_date              2478
# general_id              2478
# normative_identifier    2478
# publication_date        2478
# regulatory_authority    2478
# subject                 2478
# subject_length          2478
# subject_unique_words    2478
# subject_words           2478
# content                 2478
# text_length             2478
# text_unique_words       2478
# text_words              2478
# type                    2478
# unique_document_id      2478
# url                     2478
# new_content             2478
# dtype: int64
# 2023-10-13 12:25:04.370 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   688
# department              688
# entry_date              688
# general_id              688
# normative_identifier    688
# publication_date        688
# regulatory_authority    688
# subject                 688
# subject_length          688
# subject_unique_words    688
# subject_words           688
# content                 688
# text_length             688
# text_unique_words       688
# text_words              688
# type                    688
# unique_document_id      688
# url                     688
# new_content             688
# dtype: int64
# 2023-10-13 12:25:04.375 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   1790
# department              1790
# entry_date              1790
# general_id              1790
# normative_identifier    1790
# publication_date        1790
# regulatory_authority    1790
# subject                 1790
# subject_length          1790
# subject_unique_words    1790
# subject_words           1790
# content                 1790
# text_length             1790
# text_unique_words       1790
# text_words              1790
# type                    1790
# unique_document_id      1790
# url                     1790
# new_content             1790
# dtype: int64
# 2023-10-13 12:32:53.403 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-13 12:32:53.405 | INFO     | __main__:training_model:214 - Training loss: 0.6319377836585045
# 2023-10-13 12:33:22.719 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5456359350859229, [[ 0.29649708  0.25773552]
 
# 2023-10-13 12:33:22.723 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.40798226164079826
# 2023-10-13 12:33:22.724 | INFO     | __main__:training_model:220 - Validation loss: 0.5456359350859229
# 2023-10-13 12:33:22.725 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.40798226164079826
# 2023-10-13 12:35:45.052 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-13 12:35:45.053 | INFO     | __main__:training_model:214 - Training loss: 0.522864457257092
# 2023-10-13 12:36:14.453 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.541645841033601, [[-0.20286909  0.76568156]
 
# 2023-10-13 12:36:14.459 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5445420326223338
# 2023-10-13 12:36:14.461 | INFO     | __main__:training_model:220 - Validation loss: 0.541645841033601
# 2023-10-13 12:36:14.462 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5445420326223338
# 2023-10-13 12:38:36.871 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-13 12:38:36.872 | INFO     | __main__:training_model:214 - Training loss: 0.4511613555252552
# 2023-10-13 12:39:06.284 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4094061398350481, [[ 0.03894448  0.6488488 ]
 
# 2023-10-13 12:39:06.288 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6435100548446069
# 2023-10-13 12:39:06.289 | INFO     | __main__:training_model:220 - Validation loss: 0.4094061398350481
# 2023-10-13 12:39:06.290 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6435100548446069
# 2023-10-13 12:41:31.869 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-13 12:41:31.870 | INFO     | __main__:training_model:214 - Training loss: 0.3754444937966764
# 2023-10-13 12:42:01.235 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.39674868893378706, [[ 0.01429424  0.7755251 ]
 
# 2023-10-13 12:42:01.238 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6503759398496242
# 2023-10-13 12:42:01.240 | INFO     | __main__:training_model:220 - Validation loss: 0.39674868893378706
# 2023-10-13 12:42:01.241 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6503759398496242
# 2023-10-13 12:44:26.120 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-13 12:44:26.121 | INFO     | __main__:training_model:214 - Training loss: 0.3219384340196848
# 2023-10-13 12:44:55.514 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4039036685517475, [[ 0.12511945  0.7550211 ]
 
# 2023-10-13 12:44:55.518 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6513409961685823
# 2023-10-13 12:44:55.519 | INFO     | __main__:training_model:220 - Validation loss: 0.4039036685517475
# 2023-10-13 12:44:55.520 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6513409961685823
# 2023-10-13 12:45:29.180 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4039036685517475, [[ 0.12511945  0.7550211 ]
 
# 2023-10-13 12:45:29.182 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-13 12:45:29.183 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 640/717
# 2023-10-13 12:45:29.183 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8926080892608089

# 2023-10-13 12:45:29.184 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-13 12:45:29.185 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 170/275
# 2023-10-13 12:45:29.186 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6181818181818182

# 2023-10-13 12:45:29.194 | INFO     | __main__:results:157 - {'0': {'precision': 0.8590604026845637, 'recall': 0.8926080892608089, 'f1-score': 0.8755129958960328, 'support': 717}, '1': {'precision': 0.6882591093117408, 'recall': 0.6181818181818182, 'f1-score': 0.6513409961685823, 'support': 275}, 'accuracy': 0.8165322580645161, 'macro avg': {'precision': 0.7736597559981523, 'recall': 0.7553949537213136, 'f1-score': 0.7634269960323076, 'support': 992}, 'weighted avg': {'precision': 0.8117112538160897, 'recall': 0.8165322580645161, 'f1-score': 0.8133685403264271, 'support': 992}}
# 2023-10-13 12:45:29.196 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8590604026845637, 'recall': 0.8926080892608089, 'f1-score': 0.8755129958960328, 'support': 717}, '1': {'precision': 0.6882591093117408, 'recall': 0.6181818181818182, 'f1-score': 0.6513409961685823, 'support': 275}, 'accuracy': 0.8165322580645161, 'macro avg': {'precision': 0.7736597559981523, 'recall': 0.7553949537213136, 'f1-score': 0.7634269960323076, 'support': 992}, 'weighted avg': {'precision': 0.8117112538160897, 'recall': 0.8165322580645161, 'f1-score': 0.8133685403264271, 'support': 992}}
# 2023-10-13 12:45:29.201 | DEBUG    | __main__:<module>:2 - Department: DPT_2
# 2023-10-13 12:45:54.040 | DEBUG    | __main__:<module>:37 - Department count samples: label                   5917
# department              5917
# entry_date              5917
# general_id              5917
# normative_identifier    5917
# publication_date        5917
# regulatory_authority    5917
# subject                 5917
# subject_length          5917
# subject_unique_words    5917
# subject_words           5917
# content                 5917
# text_length             5917
# text_unique_words       5917
# text_words              5917
# type                    5917
# unique_document_id      5917
# url                     5917
# new_content             5917
# dtype: int64
# 2023-10-13 12:45:54.045 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   683
# department              683
# entry_date              683
# general_id              683
# normative_identifier    683
# publication_date        683
# regulatory_authority    683
# subject                 683
# subject_length          683
# subject_unique_words    683
# subject_words           683
# content                 683
# text_length             683
# text_unique_words       683
# text_words              683
# type                    683
# unique_document_id      683
# url                     683
# new_content             683
# dtype: int64
# 2023-10-13 12:45:54.052 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   5234
# department              5234
# entry_date              5234
# general_id              5234
# normative_identifier    5234
# publication_date        5234
# regulatory_authority    5234
# subject                 5234
# subject_length          5234
# subject_unique_words    5234
# subject_words           5234
# content                 5234
# text_length             5234
# text_unique_words       5234
# text_words              5234
# type                    5234
# unique_document_id      5234
# url                     5234
# new_content             5234
# dtype: int64
# 2023-10-13 12:58:10.208 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-13 12:58:10.210 | INFO     | __main__:training_model:214 - Training loss: 0.32919110542454044
# 2023-10-13 12:59:20.430 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.23295330428636243, [[ 2.2455244  -1.0589315 ]
 
# 2023-10-13 12:59:20.435 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5596707818930041
# 2023-10-13 12:59:20.437 | INFO     | __main__:training_model:220 - Validation loss: 0.23295330428636243
# 2023-10-13 12:59:20.438 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5596707818930041
# 2023-10-13 13:04:55.626 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-13 13:04:55.628 | INFO     | __main__:training_model:214 - Training loss: 0.22254532710000685
# 2023-10-13 13:06:05.844 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2294003208156062, [[ 2.2070613  -0.81810355]
 
# 2023-10-13 13:06:05.848 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5679513184584178
# 2023-10-13 13:06:05.850 | INFO     | __main__:training_model:220 - Validation loss: 0.2294003208156062
# 2023-10-13 13:06:05.851 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5679513184584178
# 2023-10-13 13:11:41.232 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-13 13:11:41.233 | INFO     | __main__:training_model:214 - Training loss: 0.16666356301650342
# 2023-10-13 13:12:51.485 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.25447551957832626, [[ 3.3189864  -1.8892944 ]
 
# 2023-10-13 13:12:51.489 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6116322701688555
# 2023-10-13 13:12:51.490 | INFO     | __main__:training_model:220 - Validation loss: 0.25447551957832626
# 2023-10-13 13:12:51.492 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6116322701688555
# 2023-10-13 13:18:27.040 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-13 13:18:27.041 | INFO     | __main__:training_model:214 - Training loss: 0.12739376064521588
# 2023-10-13 13:19:37.265 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.29513776587628865, [[ 3.7875211  -2.3017688 ]
 
# 2023-10-13 13:19:37.269 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6153846153846154
# 2023-10-13 13:19:37.270 | INFO     | __main__:training_model:220 - Validation loss: 0.29513776587628865
# 2023-10-13 13:19:37.271 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6153846153846154
# 2023-10-13 13:25:12.377 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-13 13:25:12.379 | INFO     | __main__:training_model:214 - Training loss: 0.0868350456671137
# 2023-10-13 13:26:22.602 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.30327236744253955, [[ 3.6732564  -2.216784  ]
 
# 2023-10-13 13:26:22.606 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6353790613718411
# 2023-10-13 13:26:22.608 | INFO     | __main__:training_model:220 - Validation loss: 0.30327236744253955
# 2023-10-13 13:26:22.609 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6353790613718411
# 2023-10-13 13:27:36.961 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.30327236744253955, [[ 3.6732564  -2.216784  ]
 
# 2023-10-13 13:27:36.963 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-13 13:27:36.964 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 1989/2094
# 2023-10-13 13:27:36.965 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9498567335243553

# 2023-10-13 13:27:36.966 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-13 13:27:36.967 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 176/273
# 2023-10-13 13:27:36.967 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6446886446886447

# 2023-10-13 13:27:36.977 | INFO     | __main__:results:157 - {'0': {'precision': 0.9534995206136145, 'recall': 0.9498567335243553, 'f1-score': 0.9516746411483252, 'support': 2094}, '1': {'precision': 0.6263345195729537, 'recall': 0.6446886446886447, 'f1-score': 0.6353790613718411, 'support': 273}, 'accuracy': 0.9146599070553443, 'macro avg': {'precision': 0.7899170200932841, 'recall': 0.7972726891065001, 'f1-score': 0.7935268512600832, 'support': 2367}, 'weighted avg': {'precision': 0.9157656611779997, 'recall': 0.9146599070553443, 'f1-score': 0.9151944158509108, 'support': 2367}}
# 2023-10-13 13:27:36.980 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9534995206136145, 'recall': 0.9498567335243553, 'f1-score': 0.9516746411483252, 'support': 2094}, '1': {'precision': 0.6263345195729537, 'recall': 0.6446886446886447, 'f1-score': 0.6353790613718411, 'support': 273}, 'accuracy': 0.9146599070553443, 'macro avg': {'precision': 0.7899170200932841, 'recall': 0.7972726891065001, 'f1-score': 0.7935268512600832, 'support': 2367}, 'weighted avg': {'precision': 0.9157656611779997, 'recall': 0.9146599070553443, 'f1-score': 0.9151944158509108, 'support': 2367}}
# 2023-10-13 13:27:36.985 | DEBUG    | __main__:<module>:2 - Department: DPT_9
# 2023-10-13 13:27:51.101 | DEBUG    | __main__:<module>:37 - Department count samples: label                   4219
# department              4219
# entry_date              4219
# general_id              4219
# normative_identifier    4219
# publication_date        4219
# regulatory_authority    4219
# subject                 4219
# subject_length          4219
# subject_unique_words    4219
# subject_words           4219
# content                 4219
# text_length             4219
# text_unique_words       4219
# text_words              4219
# type                    4219
# unique_document_id      4219
# url                     4219
# new_content             4219
# dtype: int64
# 2023-10-13 13:27:51.105 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   600
# department              600
# entry_date              600
# general_id              600
# normative_identifier    600
# publication_date        600
# regulatory_authority    600
# subject                 600
# subject_length          600
# subject_unique_words    600
# subject_words           600
# content                 600
# text_length             600
# text_unique_words       600
# text_words              600
# type                    600
# unique_document_id      600
# url                     600
# new_content             600
# dtype: int64
# 2023-10-13 13:27:51.111 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   3619
# department              3619
# entry_date              3619
# general_id              3619
# normative_identifier    3619
# publication_date        3619
# regulatory_authority    3619
# subject                 3619
# subject_length          3619
# subject_unique_words    3619
# subject_words           3619
# content                 3619
# text_length             3619
# text_unique_words       3619
# text_words              3619
# type                    3619
# unique_document_id      3619
# url                     3619
# new_content             3619
# dtype: int64
# 2023-10-13 13:35:28.728 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-13 13:35:28.729 | INFO     | __main__:training_model:214 - Training loss: 0.42225669659453735
# 2023-10-13 13:36:18.856 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.34899199997956776, [[ 0.0657659   0.32071352]
 
# 2023-10-13 13:36:18.860 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.4935064935064935
# 2023-10-13 13:36:18.861 | INFO     | __main__:training_model:220 - Validation loss: 0.34899199997956776
# 2023-10-13 13:36:18.862 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.4935064935064935
# 2023-10-13 13:40:19.937 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-13 13:40:19.938 | INFO     | __main__:training_model:214 - Training loss: 0.2777414540787773
# 2023-10-13 13:41:09.951 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2979376338117352, [[-0.04986529  0.32587084]
 
# 2023-10-13 13:41:09.955 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5781818181818182
# 2023-10-13 13:41:09.957 | INFO     | __main__:training_model:220 - Validation loss: 0.2979376338117352
# 2023-10-13 13:41:09.957 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5781818181818182
# 2023-10-13 13:45:09.803 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-13 13:45:09.805 | INFO     | __main__:training_model:214 - Training loss: 0.2273561894540956
# 2023-10-13 13:45:59.899 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3256578712015355, [[-0.33011287  0.7629442 ]
 
# 2023-10-13 13:45:59.904 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6048
# 2023-10-13 13:45:59.905 | INFO     | __main__:training_model:220 - Validation loss: 0.3256578712015355
# 2023-10-13 13:45:59.906 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6048
# 2023-10-13 13:50:00.432 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-13 13:50:00.434 | INFO     | __main__:training_model:214 - Training loss: 0.18610287872391076
# 2023-10-13 13:50:50.480 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2968502541956599, [[-0.35411483  0.76257956]
 
# 2023-10-13 13:50:50.488 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6104868913857677
# 2023-10-13 13:50:50.490 | INFO     | __main__:training_model:220 - Validation loss: 0.2968502541956599
# 2023-10-13 13:50:50.491 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6104868913857677
# 2023-10-13 13:54:51.032 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-13 13:54:51.033 | INFO     | __main__:training_model:214 - Training loss: 0.15890569253080694
# 2023-10-13 13:55:41.080 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.31362380925462935, [[-0.6514656   1.0532534 ]
 
# 2023-10-13 13:55:41.084 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6189624329159213
# 2023-10-13 13:55:41.085 | INFO     | __main__:training_model:220 - Validation loss: 0.31362380925462935
# 2023-10-13 13:55:41.086 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6189624329159213
# 2023-10-13 13:56:35.356 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.31362380925462935, [[-0.6514656   1.0532534 ]
 
# 2023-10-13 13:56:35.359 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-13 13:56:35.361 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 1302/1448
# 2023-10-13 13:56:35.362 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.899171270718232

# 2023-10-13 13:56:35.363 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-13 13:56:35.364 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 173/240
# 2023-10-13 13:56:35.365 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.7208333333333333

# 2023-10-13 13:56:35.374 | INFO     | __main__:results:157 - {'0': {'precision': 0.9510591672753835, 'recall': 0.899171270718232, 'f1-score': 0.9243876464323748, 'support': 1448}, '1': {'precision': 0.542319749216301, 'recall': 0.7208333333333333, 'f1-score': 0.6189624329159213, 'support': 240}, 'accuracy': 0.8738151658767772, 'macro avg': {'precision': 0.7466894582458423, 'recall': 0.8100023020257827, 'f1-score': 0.7716750396741481, 'support': 1688}, 'weighted avg': {'precision': 0.8929445580726704, 'recall': 0.8738151658767772, 'f1-score': 0.8809622606243483, 'support': 1688}}
# 2023-10-13 13:56:35.377 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9510591672753835, 'recall': 0.899171270718232, 'f1-score': 0.9243876464323748, 'support': 1448}, '1': {'precision': 0.542319749216301, 'recall': 0.7208333333333333, 'f1-score': 0.6189624329159213, 'support': 240}, 'accuracy': 0.8738151658767772, 'macro avg': {'precision': 0.7466894582458423, 'recall': 0.8100023020257827, 'f1-score': 0.7716750396741481, 'support': 1688}, 'weighted avg': {'precision': 0.8929445580726704, 'recall': 0.8738151658767772, 'f1-score': 0.8809622606243483, 'support': 1688}}
# 2023-10-13 13:56:35.382 | DEBUG    | __main__:<module>:2 - Department: DPT_5
# 2023-10-13 13:56:45.105 | DEBUG    | __main__:<module>:37 - Department count samples: label                   3128
# department              3128
# entry_date              3128
# general_id              3128
# normative_identifier    3128
# publication_date        3128
# regulatory_authority    3128
# subject                 3128
# subject_length          3128
# subject_unique_words    3128
# subject_words           3128
# content                 3128
# text_length             3128
# text_unique_words       3128
# text_words              3128
# type                    3128
# unique_document_id      3128
# url                     3128
# new_content             3128
# dtype: int64
# 2023-10-13 13:56:45.109 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   534
# department              534
# entry_date              534
# general_id              534
# normative_identifier    534
# publication_date        534
# regulatory_authority    534
# subject                 534
# subject_length          534
# subject_unique_words    534
# subject_words           534
# content                 534
# text_length             534
# text_unique_words       534
# text_words              534
# type                    534
# unique_document_id      534
# url                     534
# new_content             534
# dtype: int64
# 2023-10-13 13:56:45.114 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   2594
# department              2594
# entry_date              2594
# general_id              2594
# normative_identifier    2594
# publication_date        2594
# regulatory_authority    2594
# subject                 2594
# subject_length          2594
# subject_unique_words    2594
# subject_words           2594
# content                 2594
# text_length             2594
# text_unique_words       2594
# text_words              2594
# type                    2594
# unique_document_id      2594
# url                     2594
# new_content             2594
# dtype: int64
# 2023-10-13 14:02:31.582 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-13 14:02:31.583 | INFO     | __main__:training_model:214 - Training loss: 0.46653545136371305
# 2023-10-13 14:03:08.652 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3992205038888469, [[ 1.9023525  -0.7000952 ]
 
# 2023-10-13 14:03:08.656 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.19246861924686193
# 2023-10-13 14:03:08.658 | INFO     | __main__:training_model:220 - Validation loss: 0.3992205038888469
# 2023-10-13 14:03:08.659 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.19246861924686193
# 2023-10-13 14:06:09.456 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-13 14:06:09.458 | INFO     | __main__:training_model:214 - Training loss: 0.3287612990372711
# 2023-10-13 14:06:46.536 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3367696335827488, [[ 1.0369354  -0.8437185 ]
 
# 2023-10-13 14:06:46.540 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.4986149584487534
# 2023-10-13 14:06:46.541 | INFO     | __main__:training_model:220 - Validation loss: 0.3367696335827488
# 2023-10-13 14:06:46.543 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.4986149584487534
# 2023-10-13 14:09:47.285 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-13 14:09:47.287 | INFO     | __main__:training_model:214 - Training loss: 0.2567452971365244
# 2023-10-13 14:10:24.386 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.33955039229199646, [[ 1.5504177  -0.89510566]
 
# 2023-10-13 14:10:24.390 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5141509433962265
# 2023-10-13 14:10:24.391 | INFO     | __main__:training_model:220 - Validation loss: 0.33955039229199646
# 2023-10-13 14:10:24.392 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5141509433962265
# 2023-10-13 14:13:25.040 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-13 14:13:25.042 | INFO     | __main__:training_model:214 - Training loss: 0.18602355522295785
# 2023-10-13 14:14:02.136 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3808793475091945, [[ 1.8603698  -1.7701024 ]
 
# 2023-10-13 14:14:02.140 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5404255319148936
# 2023-10-13 14:14:02.141 | INFO     | __main__:training_model:220 - Validation loss: 0.3808793475091945
# 2023-10-13 14:14:02.143 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5404255319148936
# 2023-10-13 14:17:02.894 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-13 14:17:02.895 | INFO     | __main__:training_model:214 - Training loss: 0.13076902596309545
# 2023-10-13 14:17:39.997 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4035512318479873, [[ 2.4906285 -2.0392408]
 
# 2023-10-13 14:17:40.000 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5635593220338982
# 2023-10-13 14:17:40.001 | INFO     | __main__:training_model:220 - Validation loss: 0.4035512318479873
# 2023-10-13 14:17:40.002 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5635593220338982
# 2023-10-13 14:18:21.248 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4035512318479873, [[ 2.4906285 -2.0392408]
 
# 2023-10-13 14:18:21.250 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-13 14:18:21.250 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 913/1038
# 2023-10-13 14:18:21.251 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8795761078998073

# 2023-10-13 14:18:21.252 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-13 14:18:21.253 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 133/214
# 2023-10-13 14:18:21.253 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6214953271028038

# 2023-10-13 14:18:21.262 | INFO     | __main__:results:157 - {'0': {'precision': 0.9185110663983903, 'recall': 0.8795761078998073, 'f1-score': 0.8986220472440944, 'support': 1038}, '1': {'precision': 0.5155038759689923, 'recall': 0.6214953271028038, 'f1-score': 0.5635593220338982, 'support': 214}, 'accuracy': 0.8354632587859425, 'macro avg': {'precision': 0.7170074711836913, 'recall': 0.7505357175013055, 'f1-score': 0.7310906846389964, 'support': 1252}, 'weighted avg': {'precision': 0.8496264507818638, 'recall': 0.8354632587859425, 'f1-score': 0.8413509424557701, 'support': 1252}}
# 2023-10-13 14:18:21.265 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9185110663983903, 'recall': 0.8795761078998073, 'f1-score': 0.8986220472440944, 'support': 1038}, '1': {'precision': 0.5155038759689923, 'recall': 0.6214953271028038, 'f1-score': 0.5635593220338982, 'support': 214}, 'accuracy': 0.8354632587859425, 'macro avg': {'precision': 0.7170074711836913, 'recall': 0.7505357175013055, 'f1-score': 0.7310906846389964, 'support': 1252}, 'weighted avg': {'precision': 0.8496264507818638, 'recall': 0.8354632587859425, 'f1-score': 0.8413509424557701, 'support': 1252}}
# 2023-10-13 14:18:21.270 | DEBUG    | __main__:<module>:2 - Department: DPT_16
# 2023-10-13 14:18:31.090 | DEBUG    | __main__:<module>:37 - Department count samples: label                   1997
# department              1997
# entry_date              1997
# general_id              1997
# normative_identifier    1997
# publication_date        1997
# regulatory_authority    1997
# subject                 1997
# subject_length          1997
# subject_unique_words    1997
# subject_words           1997
# content                 1997
# text_length             1997
# text_unique_words       1997
# text_words              1997
# type                    1997
# unique_document_id      1997
# url                     1997
# new_content             1997
# dtype: int64
# 2023-10-13 14:18:31.094 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   439
# department              439
# entry_date              439
# general_id              439
# normative_identifier    439
# publication_date        439
# regulatory_authority    439
# subject                 439
# subject_length          439
# subject_unique_words    439
# subject_words           439
# content                 439
# text_length             439
# text_unique_words       439
# text_words              439
# type                    439
# unique_document_id      439
# url                     439
# new_content             439
# dtype: int64
# 2023-10-13 14:18:31.098 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   1558
# department              1558
# entry_date              1558
# general_id              1558
# normative_identifier    1558
# publication_date        1558
# regulatory_authority    1558
# subject                 1558
# subject_length          1558
# subject_unique_words    1558
# subject_words           1558
# content                 1558
# text_length             1558
# text_unique_words       1558
# text_words              1558
# type                    1558
# unique_document_id      1558
# url                     1558
# new_content             1558
# dtype: int64
# 2023-10-13 14:23:02.504 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-13 14:23:02.506 | INFO     | __main__:training_model:214 - Training loss: 0.5544285763055086
# 2023-10-13 14:23:26.134 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4679702785969884, [[ 0.5997205  -0.51304543]
 
# 2023-10-13 14:23:26.137 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.2986425339366516
# 2023-10-13 14:23:26.138 | INFO     | __main__:training_model:220 - Validation loss: 0.4679702785969884
# 2023-10-13 14:23:26.139 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.2986425339366516
# 2023-10-13 14:25:23.986 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-13 14:25:23.987 | INFO     | __main__:training_model:214 - Training loss: 0.42873115818947555
# 2023-10-13 14:25:47.649 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4106600350803799, [[ 0.63741016 -0.45071247]
 
# 2023-10-13 14:25:47.652 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.44106463878326996
# 2023-10-13 14:25:47.654 | INFO     | __main__:training_model:220 - Validation loss: 0.4106600350803799
# 2023-10-13 14:25:47.654 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.44106463878326996
# 2023-10-13 14:27:46.171 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-13 14:27:46.173 | INFO     | __main__:training_model:214 - Training loss: 0.3716800808906555
# 2023-10-13 14:28:09.812 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4164177241769654, [[ 1.2845447  -0.49266267]
 
# 2023-10-13 14:28:09.815 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5501618122977346
# 2023-10-13 14:28:09.817 | INFO     | __main__:training_model:220 - Validation loss: 0.4164177241769654
# 2023-10-13 14:28:09.818 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5501618122977346
# 2023-10-13 14:30:06.677 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-13 14:30:06.678 | INFO     | __main__:training_model:214 - Training loss: 0.3132689494639635
# 2023-10-13 14:30:30.348 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3712710787507671, [[ 0.6558199  -0.00737187]
 
# 2023-10-13 14:30:30.352 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6176470588235294
# 2023-10-13 14:30:30.353 | INFO     | __main__:training_model:220 - Validation loss: 0.3712710787507671
# 2023-10-13 14:30:30.354 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6176470588235294
# 2023-10-13 14:32:26.050 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-13 14:32:26.051 | INFO     | __main__:training_model:214 - Training loss: 0.27023599497042594
# 2023-10-13 14:32:49.740 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3899631259307541, [[ 1.0709275e+00 -2.4540643e-01]
 
# 2023-10-13 14:32:49.743 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5981308411214953
# 2023-10-13 14:32:49.744 | INFO     | __main__:training_model:220 - Validation loss: 0.3899631259307541
# 2023-10-13 14:32:49.745 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5981308411214953
# 2023-10-13 14:33:17.474 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3899631259307541, [[ 1.0709275e+00 -2.4540643e-01]
 
# 2023-10-13 14:33:17.476 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-13 14:33:17.477 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 574/623
# 2023-10-13 14:33:17.477 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9213483146067416

# 2023-10-13 14:33:17.478 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-13 14:33:17.479 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 96/176
# 2023-10-13 14:33:17.480 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.5454545454545454

# 2023-10-13 14:33:17.487 | INFO     | __main__:results:157 - {'0': {'precision': 0.8776758409785933, 'recall': 0.9213483146067416, 'f1-score': 0.8989819890368049, 'support': 623}, '1': {'precision': 0.6620689655172414, 'recall': 0.5454545454545454, 'f1-score': 0.5981308411214953, 'support': 176}, 'accuracy': 0.8385481852315394, 'macro avg': {'precision': 0.7698724032479174, 'recall': 0.7334014300306435, 'f1-score': 0.7485564150791502, 'support': 799}, 'weighted avg': {'precision': 0.8301829622787211, 'recall': 0.8385481852315394, 'f1-score': 0.8327118988827442, 'support': 799}}
# 2023-10-13 14:33:17.490 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8776758409785933, 'recall': 0.9213483146067416, 'f1-score': 0.8989819890368049, 'support': 623}, '1': {'precision': 0.6620689655172414, 'recall': 0.5454545454545454, 'f1-score': 0.5981308411214953, 'support': 176}, 'accuracy': 0.8385481852315394, 'macro avg': {'precision': 0.7698724032479174, 'recall': 0.7334014300306435, 'f1-score': 0.7485564150791502, 'support': 799}, 'weighted avg': {'precision': 0.8301829622787211, 'recall': 0.8385481852315394, 'f1-score': 0.8327118988827442, 'support': 799}}
